In [1]:
import os
from dotenv import load_dotenv
import sys


In [2]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [3]:
sys.path.append("/home/lillian/Documents/TenAcademy/enterprise-rag-exercise/backend")

In [14]:
from retrival import file_reader
from retrival import test_contextualized_answer

In [5]:
import pinecone


pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or '3306f52a-a64a-46dd-b81a-0d073fb5a072',
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
)

/home/lillian/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pinecone.Index(index_name)
    

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

In [8]:
context_message = file_reader("/home/lillian/Documents/TenAcademy/enterprise-rag-exercise/backend/context.txt")
document = context_message.split('\n')


NameError: name 'file_reader' is not defined

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer()

In [12]:
context_vector = vectorizer.fit_transform(document)

In [30]:
print(context_vector)

  (0, 413)	0.7266547071407753
  (0, 99)	0.6870028650523621
  (1, 138)	0.10080123133878717
  (1, 583)	0.10855373183519164
  (1, 347)	0.07778124866569268
  (1, 10)	0.10080123133878717
  (1, 579)	0.10855373183519164
  (1, 672)	0.09530074470038043
  (1, 351)	0.10855373183519164
  (1, 193)	0.10855373183519164
  (1, 404)	0.10855373183519164
  (1, 236)	0.09530074470038043
  (1, 115)	0.10855373183519164
  (1, 183)	0.10855373183519164
  (1, 55)	0.17509648840795186
  (1, 15)	0.10855373183519164
  (1, 240)	0.10855373183519164
  (1, 168)	0.10855373183519164
  (1, 205)	0.10080123133878717
  (1, 421)	0.10855373183519164
  (1, 173)	0.10080123133878717
  (1, 219)	0.10855373183519164
  (1, 539)	0.10855373183519164
  (1, 450)	0.10855373183519164
  (1, 455)	0.10855373183519164
  :	:
  (207, 490)	0.2995873471089458
  (207, 385)	0.2995873471089458
  (207, 237)	0.2995873471089458
  (207, 225)	0.2995873471089458
  (207, 460)	0.2995873471089458
  (207, 522)	0.2630116606736323
  (207, 93)	0.27819194210615195
 

In [20]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,):
    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

    

In [22]:
import openai
import IPython
response = openai.chat.completions.create(
model = "gpt-3.5-turbo",
messages = [
    {
        "role": "user",
        "content": "What is 10 academy"
    }
],
temperature = 1.0,
max_tokens = 256,
top_p =1,
frequency_penalty = 0)
print(response.choices[0].message.content)



10 Academy is a nonprofit organization that aims to provide free, world-class data science training to talented individuals from developing countries. The academy offers a rigorous 10-month program that combines online coursework with weekly virtual meetings and project-based learning. The goal of 10 Academy is to empower individuals with the skills and knowledge needed to address social, economic, and environmental challenges in their communities using data-driven insights. Upon completion of the program, graduates are connected with job opportunities and mentorship to further support their career development.


In [25]:
#PROMPTING EXPERIMENT
#few-shot prompts
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = messages
response = openai.chat.completions.create(
model = "gpt-3.5-turbo",
messages = [
    {
        "role": "user",
        "content": "What is 10 academy"
    }
],
temperature = 1.0,
max_tokens = 256,
top_p =1,
frequency_penalty = 0) 
IPython.display.Markdown(response.choices[0].message.content)

10 Academy is a data science fellowship program that aims to train and mentor aspiring data scientists from Africa. It provides a six-month intensive training program in data science and machine learning, equipping participants with the skills and knowledge needed to solve real-world problems using data. The program also focuses on building a community of data scientists in Africa and fostering collaboration and knowledge sharing. Through a combination of online learning, bootcamps, and project-based work, 10 Academy prepares fellows for careers in data science and helps them contribute to the development of their communities.